# Дистилляция [SegFormer](https://huggingface.co/docs/transformers/model_doc/segformer) (50 баллов)

Будем дистиллировать [SegFormer](https://huggingface.co/docs/transformers/model_doc/segformer) для [задачи сегментации людей](https://www.kaggle.com/datasets/laurentmih/aisegmentcom-matting-human-datasets).

## Скачаем вспомогательный код и чекпоинт бейзлайна (модели-учителя)

Скачайте архив, распакуйте, распакованные файлы выложите рядом с этим ноутбуком:
https://docs.google.com/uc?export=download&id=1J2fvxN3AJoCXkp8v_4cFjblBWkSw1VtY

### Скачаем датасет

Датасет находится по ссылке https://drive.google.com/file/d/1YOEDzZvhLb2DS1Yn7p7MSs41ou3ZBXUq/view?usp=sharing

Нужно его скачать и распаковать в папке, в которой находится ноутбук

### Установим библиотеки

In [1]:
!pip install torch transformers datasets tensorboard pillow

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/779.1 MB 36.0 MB/s eta 0:00:17^C8
     ━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.4/779.1 MB 36.2 MB/s eta 0:00:17
ERROR: Operation cancelled by user


In [2]:
import os

import typing as tp
import torch

from copy import deepcopy
from datasets import load_metric
from torch import nn
from torch.nn import functional as F
from torch.utils.tensorboard.writer import SummaryWriter
from tqdm.auto import tqdm

# utils у нас появились при скачивании вспомогательного кода. При желании можно в них провалиться-поизучать
from utils.data import init_dataloaders
from utils.model import evaluate_model
from utils.model import init_model_with_pretrain

from torch import nn
from transformers.models.segformer.modeling_segformer import SegformerLayer

In [3]:
teacher_path = 'runs/baseline_ckpt.pth'
save_dir = 'runs/distillation'

In [4]:
tb_writer = SummaryWriter(save_dir)

In [5]:
# маппинг названия классов и индексов
id2label = {
    0: "background",
    1: "human",
}
label2id = {v: k for k, v in id2label.items()}

Создадим лоадеры:

In [6]:
train_dataloader, valid_dataloader = init_dataloaders(
    root_dir=".",
    batch_size=8,
    num_workers=8,
)

/home/goncharenko/work/deep_school/.deepschool/lib/python3.10/site-packages/transformers/models/segformer/feature_extraction_segformer.py:28: FutureWarning: The class SegformerFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use SegformerImageProcessor instead.
  warnings.warn(
/home/goncharenko/work/deep_school/.deepschool/lib/python3.10/site-packages/transformers/models/segformer/image_processing_segformer.py:101: FutureWarning: The `reduce_labels` parameter is deprecated and will be removed in a future version. Please use `do_reduce_labels` instead.
  warnings.warn(


Создадим модель учителя:

In [7]:
teacher_model = init_model_with_pretrain(label2id=label2id, id2label=id2label, pretrain_path=teacher_path)

/home/goncharenko/work/deep_school/.deepschool/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b0 and are newly initialized: ['decode_head.batch_norm.bias', 'decode_head.batch_norm.num_batches_tracked', 'decode_head.batch_norm.running_mean', 'decode_head.batch_norm.running_var', 'decode_head.batch_norm.weight', 'decode_head.classifier.bias', 'decode_head.classifier.weight', 'decode_head.linear_c.0.proj.bias', 'decode_head.linear_c.0.proj.weight', 'decode_head.linear_c.1.proj.bias', 'decode_head.linear_c.1.proj.weight', 'decode_head.linear_c.2.proj.bia

И сразу отвалидируем:

In [8]:
evaluate_model(teacher_model, valid_dataloader, id2label)

/home/goncharenko/work/deep_school/t_1/utils/model.py:24: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("mean_iou")
/home/goncharenko/work/deep_school/.deepschool/lib/python3.10/site-packages/datasets/load.py:752: FutureWarning: The repository for mean_iou contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/mean_iou/mean_iou.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


Mean_iou: 0.9859684250690101
Mean accuracy: 0.9929541516927755


{'mean_iou': 0.9859684250690101,
 'mean_accuracy': 0.9929541516927755,
 'overall_accuracy': 0.992935291820804,
 'per_category_iou': array([0.98610236, 0.98583449]),
 'per_category_accuracy': array([0.99129308, 0.99461522])}

## Делаем ученика (10 баллов)

### Посмотрим, как выглядит модель учителя:

In [ ]:
teacher_model

Нас интересует (block): он состоит из нескольких ModuleList. Нас интересуют первые четыре. Посмотрим на первый из них:

```
(0): ModuleList(
          (0): SegformerLayer(
            .... тут много понаписано
          )
          (1): SegformerLayer(
            .... и тут тоже много всего
        )
```

В каждом из четырёх ModuleList сидит по два `SegformerLayer`. Нужно написать функцию, которая оставит только один (последний) из них.

In [ ]:
def create_small_network(model):
    """ Оставляет только по одному SegformerLayer в каждом ModuleList"""
    ...
    
    return small_model

def n_params(model):
    return sum(p.numel() for p in model.parameters())

In [ ]:
student_model = create_small_network(deepcopy(teacher_model))

In [ ]:
# визуализируйте и убедитесь, что у вас действительно выкинуты нужные слои
student_model

## Train Loop

Напишем старый-добрый трейнлуп и добавим в него дистилляционные лоссы.

In [ ]:
from dataclasses import dataclass

@dataclass
class TrainParams:
    n_epochs: int
    lr: float
    batch_size: int
    n_workers: int
    device: torch.device

    loss_weight: float
    last_layer_loss_weight: float
    intermediate_attn_layers_weights: tp.Tuple[float, float, float, float]
    intermediate_feat_layers_weights: tp.Tuple[float, float, float, float]

    # возможно, в ваших экспериментах захотите добавить что-то ещё

#### Домашка на поиграться-поэкспериментировать, поэтому не стесняйтесь менять параметры и выбивать скоры

In [ ]:
train_params = TrainParams(
    n_epochs=3,
    lr=6e-5,
    batch_size=8,
    n_workers=8,
    device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
    loss_weight=0.5,
    last_layer_loss_weight=0.5,
    intermediate_attn_layers_weights=(0.5, 0.5, 0.5, 0.5),
    intermediate_feat_layers_weights=(0.5, 0.5, 0.5, 0.5),
)

In [ ]:
def train(
    teacher_model,
    student_model,
    train_params: TrainParams,
    student_teacher_attention_mapping,
):
    metric = load_metric('mean_iou')
    teacher_model.to(train_params.device)
    student_model.to(train_params.device)

    teacher_model.eval()

    train_dataloader, valid_dataloader = init_dataloaders(
        root_dir=".",
        batch_size=train_params.batch_size,
        num_workers=train_params.n_workers,
    )

    optimizer = torch.optim.AdamW(student_model.parameters(), lr=train_params.lr)
    step = 0
    for epoch in range(train_params.n_epochs):
        pbar = tqdm(enumerate(train_dataloader), total=len(train_dataloader))
        for idx, batch in pbar:
            student_model.train()
            # get the inputs;
            pixel_values = batch['pixel_values'].to(train_params.device)
            labels = batch['labels'].to(train_params.device)

            optimizer.zero_grad()

            # forward + backward + optimize
            student_outputs = student_model(
                pixel_values=pixel_values, 
                labels=labels, 
                output_attentions=True,
                output_hidden_states=True,
            )
            loss, student_logits = student_outputs.loss, student_outputs.logits

            # Чего это мы no_grad() при тренировке поставили?!
            with torch.no_grad():
                teacher_output = teacher_model(
                    pixel_values=pixel_values, 
                    labels=labels, 
                    output_attentions=True,
                    output_hidden_states=True,
                )


            last_layer_loss = calc_last_layer_loss(
                student_logits,
                teacher_output.logits,
                train_params.last_layer_loss_weight,
            )

            student_attentions, teacher_attentions = student_outputs.attentions, teacher_output.attentions
            student_hidden_states, teacher_hidden_states = student_outputs.hidden_states, teacher_output.hidden_states

            intermediate_layer_att_loss = calc_intermediate_layers_attn_loss(
                student_attentions,
                teacher_attentions,
                train_params.intermediate_attn_layers_weights,
                student_teacher_attention_mapping,
            )
            
            intermediate_layer_feat_loss = calc_intermediate_layers_feat_loss(
                student_hidden_states,
                teacher_hidden_states,
                train_params.intermediate_feat_layers_weights,
            )

            total_loss = loss* train_params.loss_weight + last_layer_loss
            if intermediate_layer_att_loss is not None:
                total_loss += intermediate_layer_att_loss
            
            if intermediate_layer_feat_loss is not None:
                total_loss += intermediate_layer_feat_loss

            step += 1

            total_loss.backward()
            optimizer.step()
            pbar.set_description(f'total loss: {total_loss.item():.3f}')

            for loss_value, loss_name in (
                (loss, 'loss'),
                (total_loss, 'total_loss'),
                (last_layer_loss, 'last_layer_loss'),
                (intermediate_layer_att_loss, 'intermediate_layer_att_loss'),
                (intermediate_layer_feat_loss, 'intermediate_layer_feat_loss'),
            ):
                if loss_value is None: # для выключенной дистилляции атеншенов
                    continue
                tb_writer.add_scalar(
                    tag=loss_name,
                    scalar_value=loss_value.item(),
                    global_step=step,
                )

        #после модификаций модели обязательно сохраняйте ее целиком, чтобы подгрузить ее в случае чего
        torch.save(
            {
                'model': student_model,
                'state_dict': student_model.state_dict(),
                'optimizer_state': optimizer.state_dict(),
            },
            f'{save_dir}/ckpt_{epoch}.pth',
        )

        eval_metrics = evaluate_model(student_model, valid_dataloader, id2label)

        for metric_key, metric_value in eval_metrics.items():
            if not isinstance(metric_value, float):
                continue
            tb_writer.add_scalar(
                tag=f'eval_{metric_key}',
                scalar_value=metric_value,
                global_step=epoch,
            )


### Лосс для дистилляции последних слоёв (10б)

Напишите функцию `calc_last_layer_loss` , которая считает лосс между последними слоями учителя и ученика.

In [ ]:
#  Вдруг эти парни нам где-то пригодятся...
mse_loss = nn.MSELoss()
kl_loss = nn.KLDivLoss()

def calc_last_layer_loss(student_logits, teacher_logits, weight):
    """Считаем лосс между выходами учителя и ученика"""
    

# здесь пока не обращаем внимания, чуть позже её напишем
def calc_intermediate_layers_attn_loss(student_logits, teacher_logits, weights, student_teacher_attention_mapping):
    return None

# здесь пока не обращаем внимания, чуть позже её напишем
def calc_intermediate_layers_feat_loss(student_feat, teacher_feat, weights):
    return None

### Включим-посмотрим, как учится

In [ ]:
train(
    teacher_model=teacher_model,
    student_model=deepcopy(student_model),
    train_params=train_params,
    student_teacher_attention_mapping={}, # заполним потом
)

### Лосс для дистилляции атеншн-мап (20б)

Из каждого сегформер-блока можно достать атеншн-мапы:

In [ ]:
with torch.no_grad():
    teacher_attentions = teacher_model(pixel_values=torch.ones(1, 3, 512, 512).to(train_params.device), output_attentions=True).attentions
    student_attentions = student_model(pixel_values=torch.ones(1, 3, 512, 512).to(train_params.device), output_attentions=True).attentions

In [ ]:
teacher_attentions[0].shape

In [ ]:
assert len(teacher_attentions) == 8
assert len(student_attentions) == 4

Будем дистиллировать и их!
Но у учителя у нас их целых 8, а у ученика четыре. Поэтому нужно сделать соответствие: номер какой фичемапы у ученика
будем тянуть к какому номеру фичемапы учителя.

Сделайте правильное соответствие **(10б)**:

In [ ]:
student_teacher_attention_mapping = {
    100: 200 # сотая у ученика соответствует двухсотой у учителя (ответ заведомо неправильный),
    ... # и так несколько (сколько?) соответствий
}


Теперь напишите лосс, который принимает на вход списки фичемап ученика и учителя и тянет одно к другому. Возможно, вы захотите
учитывать разные фичемапы с разными весами. Для этого воспользуйтесь `weights` **10б**

In [ ]:
def calc_intermediate_layers_attn_loss(student_attentions, teacher_attentions, weights, student_teacher_attention_mapping):
    ...
    return intermediate_kl_loss

### Лосс для дистилляции промежуточных фиче-мап (10б)

Помимо внимания, у вас также есть карты признаков, которые можно стягивать. Напишите лосс, который бы стягивал их. Вполне возможно, что стоит их стягивать с весами, и вполне возможно что эти веса будутотличаться от весов для внимания.

In [ ]:
def calc_intermediate_layers_feat_loss(student_feats, teacher_feats, weights):
    ...
    return intermediate_mse_loss

### Теперь можем тренировать со стягиванием разных фич 

In [ ]:
train(
    teacher_model=teacher_model,
    student_model=deepcopy(student_model),
    train_params=train_params,
    student_teacher_attention_mapping=student_teacher_attention_mapping,
)

## Отправка решения

Загрузите ноутбук на образовательную платформу. Если удалось плотно поэкспериментировать — расскажите, что зашло и докинула ли вам дистилляция промежуточных слоёв